In [33]:
import numpy as np
import torch.optim
from scipy.optimize import minimize, least_squares, LinearConstraint, NonlinearConstraint
import torch.nn as nn
from torch.autograd import Variable
from enum import Enum

In [34]:
#Example 1

#Excepted coeffs
w1 = -5.201
w2 = 4.789

#Points
N = 50
deviation = 0.05
noise = torch.randn(N, 1) * deviation
X = torch.randn(N, 1)
Y = w1 * X + w2 + noise

#main
mse = nn.MSELoss()
model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for i in range(1000):
    optimizer.zero_grad()
    predictions = model(X)
    loss = mse(predictions, Y)
    loss.backward()
    optimizer.step()

print(list(model.parameters())[0].data[0, 0].item())
print(list(model.parameters())[1].data[0].item())

-5.184144020080566
4.786238670349121


In [35]:
#Example 2

#Excepted coeffs
w1 = -2.34
w2 = 8.987
w3 = 118.12
w4 = 103.1

#Points
N = 100
deviation = 0.01
noise = torch.randn(N, 1) * deviation
X = torch.randn(N, 1)
Y = w1 * (X ** 3) + w2 * (X ** 2) + w3 * X + w4 + noise

#main
v1 = Variable(torch.randn(1), requires_grad=True)
v2 = Variable(torch.randn(1), requires_grad=True)
v3 = Variable(torch.randn(1), requires_grad=True)
v4 = Variable(torch.randn(1), requires_grad=True)
mse = nn.MSELoss()
optimizer = torch.optim.SGD([v1, v2, v3, v4], lr=0.01)

for i in range(1500):
    optimizer.zero_grad()
    model = v1 * (X ** 3) + v2 * (X ** 2) + v3 * X + v4
    loss = mse(model, Y)
    loss.backward()
    optimizer.step()
    # if i == 999:
    #     print(loss)

print(v1.item())
print(v2.item())
print(v3.item())
print(v4.item())

-2.285088300704956
9.04959487915039
117.9586410522461
103.04818725585938


In [36]:
Methods = Enum('Methods', ['Classic', 'Momentum', 'AdaGrad', 'RMSprop', 'Adam', 'Nesterov'])


def optimizer_handler(method, params, lr, beta_1=0.9, beta_2=0.999):
    match method:
        case Methods.Classic:
            return torch.optim.SGD(params, lr)
        case Methods.Momentum:
            return torch.optim.SGD(params, lr, beta_1)
        case Methods.AdaGrad:
            return torch.optim.Adagrad(params, lr)
        case Methods.RMSprop:
            return torch.optim.RMSprop(params, lr, alpha=beta_2)
        case Methods.Adam:
            return torch.optim.Adam(params, lr, betas=(beta_1, beta_2))
        case Methods.Nesterov:
            return torch.optim.SGD(params, lr, nesterov=True, momentum=beta_1)


class TorchLinearRegression:
    def __init__(self, T, X, Y):
        self.T = torch.zeros(len(X), len(T))
        for i in range(len(X)):
            for j in range(len(T)):
                self.T.data[i, j] = T[j](X.data[i])
        self.X = X
        self.Y = Y
        self.W = Variable(torch.randn(len(T), 1), requires_grad=True)

    def optimize(self, method=Methods.Classic, loss=nn.MSELoss(), lr=0.01, max_steps=1500):
        optimizer = optimizer_handler(method, [self.W], lr)

        for i in range(max_steps):
            optimizer.zero_grad()
            model_i = self.T.mm(self.W)
            loss_i = loss(model_i, self.Y)
            loss_i.backward()
            optimizer.step()

        return self.W

In [37]:
#Example 3

#Excepted
excepted = np.array([-2.34, 8.987, 118.12, 103.1])
M = len(excepted)

#Parameters
N = 100
deviation = 0.01
noise = torch.randn(N, 1) * deviation
powers = [(M - 1 - i) for i in range(M)]
Funcs = np.array([lambda x, i=i: (x ** powers[i]) for i in range(M)])
X1 = torch.randn(N, 1)
Y1 = sum([excepted[i] * Funcs[i](X1) for i in range(M)]) + noise

#Calculations
lin_reg = TorchLinearRegression(Funcs, X1, Y1)
received = lin_reg.optimize().detach().numpy().reshape(1, len(Funcs))[0]
print("Excepted: " + str(excepted))
print("Received: " + str(received))
print("Absolute error: " + str(np.linalg.norm(excepted - received)))
print("Relative error: " + str(np.linalg.norm(excepted - received) / np.linalg.norm(excepted)))

Excepted: [ -2.34    8.987 118.12  103.1  ]
Received: [ -2.3353214   8.985972  118.102135  103.10001  ]
Absolute error: 0.018496323208873146
Relative error: 0.00011776515203422417
